In [1]:
#-*- coding: utf-8 -*-
#!/usr/bin/env python
import pandas as pd
import numpy as np
from copy import copy
from copy import deepcopy
import fileinput
from collections import deque as Pilha
from itertools import chain # pra transformar lista de lista em lista

# lista de pilhas
                                # g0 = { a-> str, fun -> int}
                                # g1 = g0 U {a-> int, b->int}
                                # g0 U {a-> in, b->int, c->int}       
                
# Função que retorna uma lista com cada palavra do código
def splitCod(str):
    l = []
#     print(str.split())
    l.append(str.split())
    return l[0] # pra ser somente uma lista
# lc = splitCod(teste6)

topoPilhaSplit = Pilha ()
def arquivo(str):
    p = Pilha()
    topoPilha = 0
    arq = open(str, 'r')
    texto = arq.readlines()
    for linha in texto :
        p.append(linha)
        topoPilhaSplit.append(splitCod(linha))
    arq.close()
print(arquivo('saida.txt'))

listaArquivoLinha = []
def arquivoPraLinha(str):
    p = Pilha()
    arq = open(str, 'r')
    texto = arq.readlines()
    for linha in texto :
        p.append(linha)
        listaArquivoLinha.append(splitCod(linha))

    novaListaArquivoLinha = list(chain(*listaArquivoLinha)) # olhar em :  https://pt.stackoverflow.com/questions/131258/converter-uma-lista-de-listas-em-uma-s%C3%B3-lista
    arq.close()
    return novaListaArquivoLinha
lc = arquivoPraLinha('saida.txt')
# print(arquivoPraLinha('saida.txt'))    


None


In [2]:
##                                   Funções para retornar os tipos das variáveis
# variáveis do tipo int
def intCodI(str):
    lv = {}
    for i in range(len(lc)):
        if (lc[i] == 'int' and lc[i+2] == ')') or (lc[i] == 'int' and lc[i+2] == '=') or (lc[i] == 'int' and lc[i+2] == ',') or (lc[i] == 'int' and lc[i+2] == ';'):
#             print(lc[i+1])
            lv[i] = (lc[i+1], lc[i],'var') # chave, valor
    
    return lv
varInt = np.array(intCodI(lc))
# função do tipo int
def intFunc(str):
    lv = {}
    for i in range(len(lc)):
        if lc[i] == 'int' and lc[i+2] == '(':
#             print(lc[i+1])
            lv[i] = (lc[i+1], lc[i],'func') # chave, valor
    return lv
funcInt = np.array(intCodI(lc))

# variáveis do tipo string
def intCodS(str):
    ls = {}
    for i in range(len(lc)):
        if lc[i] == 'string':
#             print(lc[i])
            ls[i] = lc[i+1]
        if lc[i] == '}': # nesse teste verifico os blocos, se tiver um } é por que tem um bloco
            ls[i] = '#' 
    return ls
varString = np.array(intCodS(lc))


##                                   funções para verificar se as operações usam tipos compatíveis; 
def verificaOpAdd(str):
    varIntOp = []
    for i in range(len(lc)):
        if lc[i] == '+':
            varIntOp.append(lc[i])
            varIntOp.append(lc[i-1])
            varIntOp.append(lc[i+1])
            varIntOp.append("#")
    return varIntOp
listOpsAdd = np.array(verificaOpAdd(lc))

def verificaOpSub(str):
    varSubOp = []
    for i in range(len(lc)):
        if lc[i] == '-':
            varSubOp.append(lc[i])
            varSubOp.append(lc[i-1])
            varSubOp.append(lc[i+1])
            varSubOp.append("#")
    return varSubOp
listOpsSub = np.array(verificaOpSub(lc))

def verificaOpMul(str):
    varMulOp = []
    for i in range(len(lc)):
        if lc[i] == '-':
            varMulOp.append(lc[i])
            varMulOp.append(lc[i-1])
            varMulOp.append(lc[i+1])
            varMulOp.append("#")
    return varMulOp
listOpsMul = np.array(verificaOpMul(lc))

def verificaOpDiv(str):
    varDivOp = []
    for i in range(len(lc)):
        if lc[i] == '-':
            varDivOp.append(lc[i])
            varDivOp.append(lc[i-1])
            varDivOp.append(lc[i+1])
            varDivOp.append("#")
    return varDivOp
listOpsDiv = np.array(verificaOpDiv(lc))    

In [4]:
# restrição de que var globais tem de ser definidas no acima de todas as funções
def verifica_func_escopo(str):
    variaveis = []
    funcoes = []
    topoPilha = 0
    varCode = lc
    escopo = Pilha ()
    escopos = [escopo]         
#     print("Função")
#     print(lc)
    i = 0
    while i< len(lc):
        if (varCode[i] == "string") or varCode[i] == "int" and varCode[i+2] == "(": # ex: int X (
            if not (varCode[i] == "string") or varCode[i] == "int" and varCode[i+2] == "(":
                if (varCode[i]=="string" or varCode[i]=="int" and varCode[i+2]=="("):
                    if not (varCode[i+1], 'int') in escopos[topoPilha] or not (varCode[i+1], 'string') in escopos[topoPilha]:                    
                        escopos[topoPilha].append( (varCode[i+1], varCode[i]) ) # formato x, int
                        variaveis.append(varCode[i+1])
#                         print(variaveis)
#                         print(escopos)
                    else:
                        variaveis.append(varCode[i+1])
#                         print("- ",variaveis)
#                         print("Variável já declarada no escopo")
                        return True
#             return False
        i += 1
# restrição de que var globais tem de ser definidas no acima de todas as funções
def verifica_var_escopo(str):
    variaveis = []
    funcoes = []
    topoPilha = 0
    varCode = lc
    escopo = Pilha ()
    escopoGlobal = Pilha ()
    
    escopos = [escopo]         
    escoposG = [escopoGlobal]          
#     print("Var")
#     print(lc)
    i = 0
    while i< len(lc):
        if (varCode[i] == "string") or varCode[i] == "int" and varCode[i+2] != "(":
            if not (varCode[i+1], 'int', 'v') in escopos[topoPilha] and not (varCode[i+1], 'string', 'v') in escopos[topoPilha]:#  and not (varCode[i+2], ')') in escopos[topoPilha]:
                if ((varCode[i] == "string") or varCode[i] == "int" and varCode[i+2] != ";"): 
                    escopos[topoPilha].append( (varCode[i+1], varCode[i], 'v') ) # formato x, int não add var global primeiro
#                     variaveis.append(varCode[i+1])
#                     print(variaveis)
            else:
                variaveis.append(varCode[i+1]) # variável que está é igual ao topo
#                 print("- ",variaveis)
#                 print("Variável já declarada no escopo")
                return  True
        i += 1
    

##                               FUNÇÕES PARA VERIFICAÇÃO DE TIPOS

# int + int senao false
def operacaoInt():
    variaveisArquivo = []
    for i in range(len(topoPilhaSplit)): 
        for j in range(len(topoPilhaSplit[i])):
            if (topoPilhaSplit[i][j] == 'int' or topoPilhaSplit[i][j] == 'string'):
                variaveisArquivo.append( (topoPilhaSplit[i][j+1], topoPilhaSplit[i][j]) )
                reversoListaVar=variaveisArquivo[::-1] # pegando as variáveis que primeiro aparecem do final para o início do arquivo

    for i in range(len(topoPilhaSplit)): 
        for j in range(len(topoPilhaSplit[i])):                
            if (topoPilhaSplit[i][j] == '+' or topoPilhaSplit[i][j] == '-' or topoPilhaSplit[i][j] == '*' or topoPilhaSplit[i][j] == '/' or topoPilhaSplit[i][j] == '>=' or topoPilhaSplit[i][j] == '<=' or topoPilhaSplit[i][j] == '>' or topoPilhaSplit[i][j] == '<'):
                if ((topoPilhaSplit[i][j-1], 'int') in reversoListaVar and (topoPilhaSplit[i][j+1], 'string') in reversoListaVar) or ((topoPilhaSplit[i][j-1], 'string') in reversoListaVar and (topoPilhaSplit[i][j+1], 'int') in reversoListaVar):                    
                    print(topoPilhaSplit[i][j-1]," EO-O ",topoPilhaSplit[i][j+1])
                    return False
                else:                                                                                                                 
                    continue                                                                                                                      
#     print(reversoListaVar)
#     return True

# print(operacaoInt())

#int a = int + int senao false
def operacaoInt2():
    variaveisArquivo = []
    for i in range(len(topoPilhaSplit)): 
        for j in range(len(topoPilhaSplit[i])):
            if (topoPilhaSplit[i][j] == 'int' or topoPilhaSplit[i][j] == 'string'):
                variaveisArquivo.append( (topoPilhaSplit[i][j+1], topoPilhaSplit[i][j]) )
                reversoListaVar=variaveisArquivo[::-1] # pegando as variáveis que primeiro aparecem do final para o início do arquivo

    for i in range(len(topoPilhaSplit)): 
        for j in range(len(topoPilhaSplit[i])):                
            if (topoPilhaSplit[i][j] == '+' or topoPilhaSplit[i][j] == '-' or topoPilhaSplit[i][j] == '*' or topoPilhaSplit[i][j] == '/' or topoPilhaSplit[i][j] == '>=' or topoPilhaSplit[i][j] == '<=' or topoPilhaSplit[i][j] == '>' or topoPilhaSplit[i][j] == '<'):
                if ((topoPilhaSplit[i][j-1], 'int') in reversoListaVar and (topoPilhaSplit[i][j-3], 'string') in reversoListaVar) or ((topoPilhaSplit[i][j-2], 'int') in reversoListaVar and (topoPilhaSplit[i][j-3], 'string') in reversoListaVar):
#                     print(topoPilhaSplit[i][j-3]," EO-1 ",topoPilhaSplit[i][j-1], topoPilhaSplit[i][j+1])
                    return False                
                else:
                    continue                
    print(reversoListaVar)
#     return True

print("op int2: ",operacaoInt2())

# string a =  int
def caseStringIgualInt():
    variaveisArquivo = []
    for i in range(len(topoPilhaSplit)): 
        for j in range(len(topoPilhaSplit[i])):
            if (topoPilhaSplit[i][j] == 'int' or topoPilhaSplit[i][j] == 'string'):
                variaveisArquivo.append( (topoPilhaSplit[i][j+1], topoPilhaSplit[i][j]) )
                reversoListaVar=variaveisArquivo[::-1] # pegando as variáveis que primeiro aparecem do final para o início do arquivo

    for i in range(len(topoPilhaSplit)): 
        for j in range(len(topoPilhaSplit[i])):                

            if topoPilhaSplit[i][j] == '=':
                if topoPilhaSplit[i][j-2] == 'int':
                        continue

                # caso string recebe int: string a = 3
                elif (topoPilhaSplit[i][j+1][0] == '\"' and (topoPilhaSplit[i][j-1], 'string') in reversoListaVar) or (topoPilhaSplit[i][j+1][0] == '\'' and ((topoPilhaSplit[i][j-1], 'string') in reversoListaVar)):
#                     print(topoPilhaSplit[i][j-1]," 22 ",topoPilhaSplit[i][j+1])
                    continue 
                else:
#                     print("ERRO ",topoPilhaSplit[i][j-1]," E22 ",topoPilhaSplit[i][j+1]) 
                    return False
                
#     print(reversoListaVar)
#     return True

# print(caseStringIgualInt())

# int a  =  string
def caseIntIgualString():
    variaveisArquivo = []
    for i in range(len(topoPilhaSplit)): 
        for j in range(len(topoPilhaSplit[i])):
            if (topoPilhaSplit[i][j] == 'int' or topoPilhaSplit[i][j] == 'string'):
                variaveisArquivo.append( (topoPilhaSplit[i][j+1], topoPilhaSplit[i][j]) )
                reversoListaVar=variaveisArquivo[::-1] # pegando as variáveis que primeiro aparecem do final para o início do arquivo

    for i in range(len(topoPilhaSplit)): 
        for j in range(len(topoPilhaSplit[i])):                

            if topoPilhaSplit[i][j] == '=':
                if topoPilhaSplit[i][j-2] == 'string':
                    continue
                # caso int recebe string: int a = "3"
                elif (topoPilhaSplit[i][j+1][0] == '\"' and ((topoPilhaSplit[i][j-1], 'int') in reversoListaVar)) or (topoPilhaSplit[i][j+1][0] == '\'' and ((topoPilhaSplit[i][j-1], 'int') in reversoListaVar)):                    
#                     print("ERRO ",topoPilhaSplit[i][j-1]," E11 ",topoPilhaSplit[i][j+1], topoPilhaSplit[i][j+1][0]) 
                    return False
                else:
#                     print(topoPilhaSplit[i][j-1]," 11 ",topoPilhaSplit[i][j+1], topoPilhaSplit[i][j+1][0]  )
                    continue 
#     print(reversoListaVar)
#     return True

# print(caseIntIgualString())

# string +-*/ string é pra dá erro
def operacaoString():
    variaveisArquivo = []
    for i in range(len(topoPilhaSplit)): 
        for j in range(len(topoPilhaSplit[i])):
            if (topoPilhaSplit[i][j] == 'int' or topoPilhaSplit[i][j] == 'string'):
                variaveisArquivo.append( (topoPilhaSplit[i][j+1], topoPilhaSplit[i][j]) )
                reversoListaVar=variaveisArquivo[::-1] # pegando as variáveis que primeiro aparecem do final para o início do arquivo

    for i in range(len(topoPilhaSplit)): 
        for j in range(len(topoPilhaSplit[i])):                
            if (topoPilhaSplit[i][j] == '+' or topoPilhaSplit[i][j] == '-' or topoPilhaSplit[i][j] == '*' or topoPilhaSplit[i][j] == '/'):
                if ((topoPilhaSplit[i][j-1], 'string') in reversoListaVar and (topoPilhaSplit[i][j+1], 'string') in reversoListaVar) or ((topoPilhaSplit[i][j-1], 'string') in reversoListaVar and (topoPilhaSplit[i][j+1], 'int') in reversoListaVar):                    
#                     print(topoPilhaSplit[i][j-1]," EStrO-O ",topoPilhaSplit[i][j+1])
                    return False
                else:                                                                                                                 
                    continue                                                                                                                      
#     print(reversoListaVar)
#     return True
# print(operacaoString())



# caso string ++ e == 
def string_add():
#     print("pilha ", topoPilhaSplit) # verificando se a pilha vem caregada
    variaveisArquivo = []
    for i in range(len(topoPilhaSplit)):
        for j in range(len(topoPilhaSplit[i])):
            if (topoPilhaSplit[i][j] == 'int' or topoPilhaSplit[i][j] == 'string'):
                variaveisArquivo.append( (topoPilhaSplit[i][j+1], topoPilhaSplit[i][j]) )
                reversoListaVar=variaveisArquivo[::-1] # pegando as variáveis que primeiro aparecem do final para o início do arquivo

    for i in range(len(topoPilhaSplit)): 
        for j in range(len(topoPilhaSplit[i])):                
            if (topoPilhaSplit[i][j] == '++' or topoPilhaSplit[i][j] == '=='):
                if topoPilhaSplit[i][j-4] == 'int':
                    return False
                elif ((topoPilhaSplit[i][j-1], 'int') in reversoListaVar or (topoPilhaSplit[i][j+1], 'int') in reversoListaVar) or (not topoPilhaSplit[i][j-1][0] == '\"' and topoPilhaSplit[i][j-1][0].isdigit()) or (not topoPilhaSplit[i][j-1][0] == '\'' and topoPilhaSplit[i][j-1][0].isdigit()) or (not topoPilhaSplit[i][j+1][0] == '\"' and topoPilhaSplit[i][j+1][0].isdigit()) or (not topoPilhaSplit[i][j+1][0] == '\'' and topoPilhaSplit[i][j+1][0].isdigit()): 

#                     print(topoPilhaSplit[i][j-1]," ERRO ++ ",topoPilhaSplit[i][j+1])
                    return False
                else:           
#                     print(topoPilhaSplit[i][j-1]," ++ ",topoPilhaSplit[i][j+1])
                    continue                                                                                                                      
    
# print(string_add())


# " e ", " e ', ' e ', " e '
def operacaoStringNaoVar():
    variaveisArquivo = []
    for i in range(len(topoPilhaSplit)): 
        for j in range(len(topoPilhaSplit[i])):
            if (topoPilhaSplit[i][j] == 'int' or topoPilhaSplit[i][j] == 'string'):
                variaveisArquivo.append( (topoPilhaSplit[i][j+1], topoPilhaSplit[i][j]) )

    for i in range(len(topoPilhaSplit)): 
        for j in range(len(topoPilhaSplit[i])):                
            if (topoPilhaSplit[i][j] == '+' or topoPilhaSplit[i][j] == '-' or topoPilhaSplit[i][j] == '*' or topoPilhaSplit[i][j] == '/'):
                # " e ", " e ', ' e ', " e '
                if ((topoPilhaSplit[i][j-1][0] == '\"') and (topoPilhaSplit[i][j+1][0] == '\"')) or ((topoPilhaSplit[i][j-1][0] == '\'') and (topoPilhaSplit[i][j+1][0] == '\'')) or ((topoPilhaSplit[i][j-1][0] == '\'') and (topoPilhaSplit[i][j+1][0] == '\"')) or ((topoPilhaSplit[i][j-1][0] == '\"') and (topoPilhaSplit[i][j+1][0] == '\'')):                    
#                     print(topoPilhaSplit[i][j-1]," EO-O ",topoPilhaSplit[i][j+1])
                    return False
                else:                                                                                                                 
                    continue                                                                                                                      
#     return True
# print(operacaoStringNaoVar())
# operação int com int
def operacaoIntNaoVar():
    variaveisArquivo = []
    reversoListaVar = []
    for i in range(len(topoPilhaSplit)): 
        for j in range(len(topoPilhaSplit[i])):
            if (topoPilhaSplit[i][j] == 'int' or topoPilhaSplit[i][j] == 'string'):
                variaveisArquivo.append( (topoPilhaSplit[i][j+1], topoPilhaSplit[i][j]) )
                reversoListaVar=variaveisArquivo[::-1] # pegando as variáveis que primeiro aparecem do final para o início do arquivo
# para pegar número ou um valor que não é variável nem número basta verificar se está na lista de variáveis
# e se a posição inicial tem aspas(com isso caracterizo um valor que não é string nem número)
    for i in range(len(topoPilhaSplit)): 
        for j in range(len(topoPilhaSplit[i])):                
            if (topoPilhaSplit[i][j] == '+' or topoPilhaSplit[i][j] == '-' or topoPilhaSplit[i][j] == '*' or topoPilhaSplit[i][j] == '/' or topoPilhaSplit[i][j] == '>=' or topoPilhaSplit[i][j] == '<=' or topoPilhaSplit[i][j] == '>' or topoPilhaSplit[i][j] == '<'): # or topoPilhaSplit[i][j] == '==' or topoPilhaSplit[i][j] == '++'):
#int d; ou d = string
#"3" + 5 ; ou '3' + 5 ; ou "d" + 5 ; ou 'd' + 5 ; ou como 2° parâmetro

                if  (topoPilhaSplit[i][j-1][0] == '\"' and (topoPilhaSplit[i][j-1][1].isdigit()) and not (topoPilhaSplit[i][j-1][1],'int') in reversoListaVar) or      (topoPilhaSplit[i][j+1][0] == '\"' and (topoPilhaSplit[i][j+1][1].isdigit()) and not (topoPilhaSplit[i][j+1][1],'int') in reversoListaVar) or     (topoPilhaSplit[i][j-1][0] == '\'' and (topoPilhaSplit[i][j-1][1].isdigit()) and not (topoPilhaSplit[i][j-1][1],'int') in reversoListaVar) or      (topoPilhaSplit[i][j+1][0] == '\'' and (topoPilhaSplit[i][j+1][1].isdigit()) and not (topoPilhaSplit[i][j+1][1],'int') in reversoListaVar)  or                   (topoPilhaSplit[i][j-1][0] == '\"' and (not topoPilhaSplit[i][j-1][1].isdigit()) and not (topoPilhaSplit[i][j-1][1],'int') in reversoListaVar) or      (topoPilhaSplit[i][j+1][0] == '\"' and (not topoPilhaSplit[i][j+1][1].isdigit()) and not (topoPilhaSplit[i][j+1][1],'int') in reversoListaVar) or     (topoPilhaSplit[i][j-1][0] == '\'' and (not topoPilhaSplit[i][j-1][1].isdigit()) and not (topoPilhaSplit[i][j-1][1],'int') in reversoListaVar) or      (topoPilhaSplit[i][j+1][0] == '\'' and (not topoPilhaSplit[i][j+1][1].isdigit()) and not (topoPilhaSplit[i][j+1][1],'int') in reversoListaVar)  :
#                     print(topoPilhaSplit[i][j-1]," EO-O ",topoPilhaSplit[i][j+1])
                    return False
                else:                                                                                                                 
                    continue
#     print(reversoListaVar)
# print(operacaoIntNaoVar())

def analisadorDeTipos():
    if operacaoIntNaoVar()!=False and operacaoStringNaoVar()!=False and string_add() != False:
#         print("Sucesso") # pegar casos em que o arquivo tem somente algo do tipo
        return True
    elif operacaoIntNaoVar()!=False and operacaoStringNaoVar()!=False and caseIntIgualString() !=False and caseStringIgualInt()!=False and operacaoInt2()!=False and  operacaoInt()!=False and operacaoString()!=False and string_add() != False:
#         print("Sucesso")
        return True #"Sucesso"
    else:
#         print("Erro")
        return False# "Erro"    
# print(analisadorDeTipos())
# print(caseIntIgualString())


# III. Se as variáveis (e funções) estão sendo usadas sem serem declaradas.
def var_func_ordem_declaracao():
    variaveisArquivo = []
    funcArquivo = []
    reversoListaVar = []
    indicesFunc = []
    indicesVar = []
    
    topoPilha = 0
# Pegando todas as funções e colocando eles em uma lista na ordem que são declaradas    
    for i in range(len(topoPilhaSplit)): 
        for j in range(len(topoPilhaSplit[i])):
            if (topoPilhaSplit[i][j] == 'int' or topoPilhaSplit[i][j] == 'string') and topoPilhaSplit[i][j+2] == '(':
                    funcArquivo.append( (topoPilhaSplit[i][j+1]) )
                    indicesFunc.append((i,j))
# Pegando todas as variáveis e colocando eles em uma lista na ordem que são declaradas    
    for i in range(len(topoPilhaSplit)): 
        for j in range(len(topoPilhaSplit[i])):
            if (topoPilhaSplit[i][j] == 'int' or topoPilhaSplit[i][j] == 'string') and not topoPilhaSplit[i][j+2] == '(':
                    variaveisArquivo.append( (topoPilhaSplit[i][j+1] ) )
                    indicesVar.append((i,j))
                    reversoListaVar=variaveisArquivo[::-1] 

# Verificando se as funções estão sendo declaradas antes de serem usadas
    for i in range(len(topoPilhaSplit)): 
        for j in range(len(topoPilhaSplit[i])):
            if topoPilhaSplit[i][j] == '(':
                # pegando declarações de funções
                if not (topoPilhaSplit[i][j-2] == 'int') and not(topoPilhaSplit[i][j-2] == 'string'):
                    for k in range(len(indicesFunc)):
                        if topoPilhaSplit[i][j-1] == funcArquivo[k]: # quando encontro uma função declarada
                            if (i,j) > indicesFunc[k]: # vejo se ela e declarada antes
#                                 print((i,j), indicesFunc[k])
                                continue
                            else:
                                print("A função ", topoPilhaSplit[i][j-1], " foi declarada antes de ser usada: ", topoPilhaSplit[i][j-1], " tem índice ", (i,j), " a declaração dela tem índice ", indicesFunc[k])
                                return False

# verificando se as variáveis estão sendo declaradas antes de serem usadas
    for i in range(len(topoPilhaSplit)): 
        for j in range(len(topoPilhaSplit[i])):
            if topoPilhaSplit[i][j] != '(':
                # pegando declarações de funções
                if not (topoPilhaSplit[i][j-2] == 'int') and not(topoPilhaSplit[i][j-2] == 'string'):
                    for k in range(len(indicesVar)):
                        if topoPilhaSplit[i][j-1] == variaveisArquivo[k]: # quando encontro uma variável declarada
                            if (i,j) > indicesVar[k]: # vejo se ela e declarada antes
#                                 print((i,j), indicesVar[k])
                                continue
                            else:
#                                 print("A variável ", topoPilhaSplit[i][j-1], " foi declarada antes de ser usada: ", topoPilhaSplit[i][j-1], " tem índice ", (i,j), " a declaração dela tem índice ", indicesVar[k])
                                return False


#     print(reversoListaVar)
    

[('Xim', 'string'), ('Pun', 'string'), ('ff', 'int'), ('Pun', 'int'), ('Lun', 'string'), ('x5', 'int'), ('x4', 'int'), ('Lun', 'int'), ('v', 'int'), ('v2', 'int'), ('x44', 'string'), ('Lun', 'string'), ('c', 'string')]
op int2:  None


In [ ]:
# lc = arquivoPraLinha('saida.txt')

# retorna Erro se o parâmetro de retorno não é o mesmo do tipo da função
def tipo_funcao_mesmo_tipo_retorno_param():
    variaveisArquivo = []
    var_func_Arquivo = []
    so_tipo_var_func_Arquivo = []
    
    tiposVariaveisArquivo = []
    funcArquivo = []
    funcArquivoZeroParam = []
    funcArquivoUmParam = []
    funcArquivoDoisParam = []
    varParametrosReturn = [] # lista com os parâmetros e os tipos dos retornos
    reversoListaVar = []
    topoPilha = 0
# Pegando todas as variáveis e funções e colocando eles em uma lista na ordem que são declaradas    
    for i in range(len(topoPilhaSplit)): 
        for j in range(len(topoPilhaSplit[i])):
            if ((topoPilhaSplit[i][j] == 'int' or topoPilhaSplit[i][j] == 'string') and not topoPilhaSplit[i][j+2] == '(') or ((topoPilhaSplit[i][j] == 'int' or topoPilhaSplit[i][j] == 'string') and topoPilhaSplit[i][j+2] == '('):
                var_func_Arquivo.append( (topoPilhaSplit[i][j+1], topoPilhaSplit[i][j] ) )                    
                so_tipo_var_func_Arquivo.append(topoPilhaSplit[i][j])                    
                    
                
# Pegando todas as variáveis e colocando eles em uma lista na ordem que são declaradas    
    for i in range(len(topoPilhaSplit)): 
        for j in range(len(topoPilhaSplit[i])):
            if (topoPilhaSplit[i][j] == 'int' or topoPilhaSplit[i][j] == 'string') and not topoPilhaSplit[i][j+2] == '(':
                    variaveisArquivo.append( (topoPilhaSplit[i][j+1], topoPilhaSplit[i][j] ) )
                    tiposVariaveisArquivo.append( (topoPilhaSplit[i][j] ) )
                    
                    reversoListaVar=variaveisArquivo[::-1] 
# Pegando todas as funções com seus parâmetros e colocando eles em uma lista na ordem que são declaradas    
    for i in range(len(topoPilhaSplit)): 
        for j in range(len(topoPilhaSplit[i])):
            if (topoPilhaSplit[i][j] == 'int' or topoPilhaSplit[i][j] == 'string') and topoPilhaSplit[i][j+2] == '(':
                funcArquivo.append( (topoPilhaSplit[i][j] ) ) # pegando as funções na ordem que são declaradas
            if (topoPilhaSplit[i][j] == 'int' or topoPilhaSplit[i][j] == 'string') and topoPilhaSplit[i][j+2] == '(' and topoPilhaSplit[i][j+3] == ')': 
                funcArquivoZeroParam.append( (topoPilhaSplit[i][j]) ) # *só preciso guardar a func
            
            if ((topoPilhaSplit[i][j] == 'int' or topoPilhaSplit[i][j] == 'string') and topoPilhaSplit[i][j+2] == '(' and (topoPilhaSplit[i][j+3] == 'int' or topoPilhaSplit[i][j+3] == 'string') and topoPilhaSplit[i][j+5] ==')' ):
                    funcArquivoUmParam.append( (topoPilhaSplit[i][j]) ) # *só preciso a func

            if ((topoPilhaSplit[i][j] == 'int' or topoPilhaSplit[i][j] == 'string') and topoPilhaSplit[i][j+2] == '(' and (topoPilhaSplit[i][j+3] == 'int' or topoPilhaSplit[i][j+3] == 'string') and (topoPilhaSplit[i][j+6] == 'int' or topoPilhaSplit[i][j+6] == 'string') and topoPilhaSplit[i][j+8] ==')' ):
                    funcArquivoDoisParam.append((topoPilhaSplit[i][j])) # func
    
# Pegando os parametros do retorno das funções
    for i in range(len(topoPilhaSplit)): 
        for j in range(len(topoPilhaSplit[i])):
            if topoPilhaSplit[i][j] == 'return':
                # para return com 1 or 2 param tipo var or func
                if ((topoPilhaSplit[i][j+1], 'int') in var_func_Arquivo) and (topoPilhaSplit[i][j+2] == '+' or topoPilhaSplit[i][j+2] == '-' or topoPilhaSplit[i][j+2] == '*' or topoPilhaSplit[i][j+2] == '/' or topoPilhaSplit[i][j+2] == '++' or topoPilhaSplit[i][j+2] == '==') and topoPilhaSplit[i][j+4] == ';' or ((topoPilhaSplit[i][j+1], 'int') in var_func_Arquivo and topoPilhaSplit[i][j+2] == ';'):

# or porque caso tenha int string vai dá erro por outros métodos # retorno com um parâmetro ou dois parâmetros        
#obs: basto pegar um dos parâmetros pois se não for do mesmo tipo vai dá erro por outras operações
                    varParametrosReturn.append('int') # tipos das funções na ordem que são declaradas

                if ((topoPilhaSplit[i][j+1], 'string') in var_func_Arquivo) and (topoPilhaSplit[i][j+2] == '+' or topoPilhaSplit[i][j+2] == '-' or topoPilhaSplit[i][j+2] == '*' or topoPilhaSplit[i][j+2] == '/' or topoPilhaSplit[i][j+2] == '++' or topoPilhaSplit[i][j+2] == '==') and topoPilhaSplit[i][j+4] == ';' or ((topoPilhaSplit[i][j+1], 'string') in var_func_Arquivo and topoPilhaSplit[i][j+2] == ';'):
                    varParametrosReturn.append('string') # tipos das funções na ordem que são declaradas

# verificando se as funções de 1 parâmetro é igual ao retorno da função
    for l in range(len(funcArquivo)):
#         print("Aqui 1 ")
        if not(varParametrosReturn[l] == funcArquivo[l] ): # O caso é que se criar uma função que repare o tipo da funções com um e dois parâmetros de retorno dá 100% para todos os casos
#             print("Aqui 2 ")
#             print("Erro Int or String", varParametrosReturn[l], funcArquivo[l])
            return False
        else:
#             print("continuando ", varParametrosReturn[l], funcArquivo[l])
            continue
          

#     print("nada ",funcArquivo)                                                                                                                                
#     print("Zero ",funcArquivoZeroParam)
#     print("Um ",funcArquivoUmParam)
#     print("Dois ",funcArquivoDoisParam)
#     print("Var ",    variaveisArquivo)
#     print("Var. Func. ",    var_func_Arquivo)
    
#     print("Rev.Var ",    reversoListaVar)
#     print("Func. Arquivo ",    funcArquivo)
#     print("Var. Param. Um ",    varParametrosReturn)

#     print("Rev. Lista ",    reversoListaVar)
                                                                                    
# print(tipo_funcao_mesmo_tipo_retorno_param())


In [ ]:
# print(analisadorDeTipos())
# fazer sobrecarga de função **

In [ ]:
# O erro de nome de param com o mesmo nome e tipo que o param da variável
# Verificar também
# string a = 3 ;
# string b = 5 ;
# int c = a + b ;

# pegar esse caso
# int fun ( int a , int b ) {
# 	return a + b ;
# }
# int main ( ) {
# 	string a = 3 ;
# 	string b = 5 ;
# 	int c = a + b ;
# }

In [ ]:
def Analisador():
#     print(lc)
#     if #var_func_ordem_declaracao() != False: # erro nessa função

    if not verifica_var_escopo(lc)  and not verifica_func_escopo(lc) and analisadorDeTipos() != False and tipo_funcao_mesmo_tipo_retorno_param() !=False: # and var_func_ordem_declaracao() != False :
        return "Sucesso"
    else:
        return "Erro"

print(Analisador())
# print(teste6)

In [ ]:
# sucesso
# in 1, in 2 **ERA PRA DÁ ERRO , in 3 **ERA PRA DÁ ERRO
# teste0, teste2, teste4, teste5, teste6 ** ERRO (func def dentro de func ) , teste7 ** mesmo prob teste6, teste8, 
# teste13 
# erro
# in 0
# teste1, teste3, teste9 teste10 ** 9 e 10 era pra ser sucesso prob mesmo nome param
# teste11,teste12, ** era pra ser sucesso prob mesmo nome param

# ajeitar casos teste 9 10 12, e em sucesso in2, in3, teste6, teste 6,7,8 